In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

def check(n_point, df):
    N = n_point
    df = df
    check_constraint1 = []
    check_constraint2 = []
    check_constraint3 = []

    check_constraint1_check = False
    check_constraint2_check = False
    check_constraint3_check = False

    for k in range(len(df)):
        #c[0][0]
        for i in range(N):
            check_tempc1_1 = dimod.quicksum(df['sample'][k][f'c[{i}][{j}]'] for j in range(N) if i!=j)
            if check_tempc1_1 != 1:
                check_constraint1_check = True
        for i in range(N):
            check_tempc2_1 = dimod.quicksum(df['sample'][k][f'c[{j}][{i}]'] for j in range(N) if i!=j)
            if check_tempc2_1 != 1:
                check_constraint2_check = True
        if check_constraint1_check == True:
            check_constraint1.append(k)
            check_constraint1_check = False
        if check_constraint2_check == True:
            check_constraint2.append(k)
            check_constraint2_check = False

    num_index = []
    for i in range(len(df)):
        for c1 in check_constraint1:
            num_index.append(c1)
            #num_index = np.delete(num_index, c1)
        for c2 in check_constraint2:
            num_index.append(c2)
        for c3 in check_constraint3:
            num_index.append(c3)

    if num_index == []:
        num_index = np.unique(num_index)
        feasible_index = np.arange(0,len(df),1)
    else:
        num_index = np.unique(num_index)
        feasible_index = np.arange(0,len(df),1)
        feasible_index = np.delete(feasible_index, num_index)

    min_energy = 1e9
    min_index = 0
    for i in feasible_index:
        if (min_energy > df['energy'][i]) and (df['energy'][i]>=0):
            min_energy = df['energy'][i]
            min_index = i
    if len(feasible_index) == 0:
        done = False#True # infeasible
    else:
        done = True #False # feasible
    #print(done)
    #print(df['energy'][min_index])
    energycal = 0
    for i in range(N):
        for j in range(N):
            for k in range(N):
                    energycal += distances[i][j]*df['sample'][min_index][f'c[{i}][{k}]']*df['sample'][min_index][f'c[{j}][{(k+1)%N}]']

    success_probability = len(feasible_index)/len(df)
    return df['sample'][min_index], df['energy'][min_index], done, energycal, success_probability#df['energy'][min_index] #min_index, energycal


In [27]:
from dimod import ConstrainedQuadraticModel, CQM, Binary, quicksum, BinaryQuadraticModel, ExactCQMSolver
from dwave.samplers import SteepestDescentSampler, SimulatedAnnealingSampler
import dimod
import neal
import numpy as np
#import greedy

In [29]:
# ===== TSP (positional MIQP without y) — full run script =====
# pip install docplex scipy tqdm numpy pandas
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
from docplex.mp.model import Model
from scipy.spatial import distance_matrix
from typing import Optional

# ------------------------------------------------------------
# 0) Instance (사용자 사양 반영: 항상 np.random.seed(0), depot=(0,0))
# ------------------------------------------------------------
def TSP_instance(n_customer: int, seed: int = 0):
    """
    n_customer: 고객 수 (depot 제외). 거리행렬 크기는 (n_customer+1) x (n_customer+1)
    seed: 인자는 유지하지만, 아래 구현은 항상 np.random.seed(0)으로 고정합니다.
    """
    n_point = n_customer + 1
    assert n_point >= 1

    # 재현성: 사용자 요구대로 고정 시드(0)
    np.random.seed(0)

    # 무작위 좌표 생성 (pandas 사용)
    df = pd.DataFrame({
        'x': np.random.randint(0, 100, n_point),
        'y': np.random.randint(0, 100, n_point),
    })
    # depot (index 0) 고정
    df.loc[0, 'x'] = 0
    df.loc[0, 'y'] = 0

    D = distance_matrix(df[['x', 'y']].values, df[['x', 'y']].values)
    return np.asarray(D, dtype=float)

# ------------------------------------------------------------
# 1) MIQP positional model (no y variables)
#    - 목적함수: sum_{k,i,j} d_ij * x_{i,k} * x_{j,k+1}
#    - 제약: 각 도시 1회, 각 포지션 1개 (선형 등식)
#    - 대칭깨기(선택): x[0,0] == 1
# ------------------------------------------------------------
def solve_tsp_miqp_positional(
    distances,
    anchor_depot0: bool = True,
    timelimit: Optional[float] = None,
    mip_gap: Optional[float] = None,
    randomseed: Optional[int] = None,
    log_output: bool = False,
):
    """Minimize sum_{k,i,j} d_ij * x_{i,k} * x_{j,k+1} s.t. ∑_k x_{i,k}=1, ∑_i x_{i,k}=1."""
    D = np.asarray(distances, dtype=float)
    n = D.shape[0]
    mdl = Model(name="TSP_MIQP_Positional", log_output=log_output)

    # binary x[i,k]: city i at position k
    x = mdl.binary_var_matrix(n, n, name="x")

    # linear constraints
    for i in range(n):
        mdl.add(mdl.sum(x[i, k] for k in range(n)) == 1)   # each city once
    for k in range(n):
        mdl.add(mdl.sum(x[i, k] for i in range(n)) == 1)   # one city per position

    # break rotation symmetry (optional)
    if anchor_depot0:
        mdl.add(x[0, 0] == 1)

    # MIQP objective (no y): sum d_ij * x_{i,k} * x_{j,k+1}
    dist_term = mdl.sum(D[i, j] * x[i, k] * x[j, (k + 1) % n]
                        for k in range(n) for i in range(n) for j in range(n))
    mdl.minimize(dist_term)

    # solver params
    if timelimit is not None:
        mdl.parameters.timelimit = timelimit
    if mip_gap is not None:
        mdl.parameters.mip.tolerances.mipgap = mip_gap
    if randomseed is not None:
        mdl.parameters.randomseed = int(randomseed)

    # solve + timing
    t0 = time.time()
    sol = mdl.solve()
    elapsed_ms = (time.time() - t0) * 1000.0

    if sol is None:
        return dict(ok=False, time_ms=elapsed_ms, distance=np.nan, tour=None, model=mdl)

    # reconstruct tour by positions
    tour = []
    for k in range(n):
        vals = [sol.get_value(x[i, k]) for i in range(n)]
        tour.append(int(np.argmax(vals)))

    # route cost from D (should equal objective)
    route_cost = sum(D[tour[k], tour[(k + 1) % n]] for k in range(n))
    return dict(ok=True, time_ms=elapsed_ms, distance=route_cost, tour=tour, model=mdl)

# ------------------------------------------------------------
# 2) Repeat runner (tqdm + ms) : 같은 인스턴스를 runs회 풀어 요약
# ------------------------------------------------------------
def run_repeats_miqp(
    distances,
    runs: int = 10,
    timelimit: Optional[float] = None,
    mip_gap: Optional[float] = None,
):
    recs = []
    for j in tqdm(range(runs), desc="MIQP runs", leave=False):
        res = solve_tsp_miqp_positional(
            distances, anchor_depot0=True,
            timelimit=timelimit, mip_gap=mip_gap,
            randomseed=1234 + j, log_output=False
        )
        recs.append(res)
    ok_recs = [r for r in recs if r["ok"]]
    df = pd.DataFrame(dict(
        distance=[r["distance"] for r in ok_recs],
        time_ms=[r["time_ms"] for r in ok_recs],
    ))
    summary = dict(
        distance_mean=float(df["distance"].mean()) if not df.empty else np.nan,
        distance_std=float(df["distance"].std(ddof=1)) if len(df) > 1 else np.nan,
        time_mean_ms=float(df["time_ms"].mean()) if not df.empty else np.nan,
        time_std_ms=float(df["time_ms"].std(ddof=1)) if len(df) > 1 else np.nan,
        feasible_rate=len(ok_recs) / runs  # 제약형이라 보통 1.0 (해를 못 찾으면 0으로 반영)
    )
    return summary, df, recs  # df: per-run metrics, recs: includes tours

# ------------------------------------------------------------
# 3) Main: Single run + N=1..10 요약 저장
# ------------------------------------------------------------
if __name__ == "__main__":
    # Single run demo
    N = 7
    D = TSP_instance(N, seed=0)
    print(f"[Single run] N={N}, n_point={N+1}")
    res = solve_tsp_miqp_positional(D, anchor_depot0=True, randomseed=0)
    print("ok:", res["ok"], "distance:", res["distance"], "time_ms:", f"{res['time_ms']:.2f}")
    print("tour:", res["tour"])

    # Multiple N with 10 repeats each → summary CSV
    rows = []
    runs = 10
    for N in tqdm(range(1, 11), desc="N loop (summaries)"):  # N=1..10
        D = TSP_instance(N, seed=0)
        summary, df_runs, recs = run_repeats_miqp(D, runs=runs)
        row = dict(N=N, n_point=N+1, **summary)
        rows.append(row)

    summary_df = pd.DataFrame(rows).sort_values("N").reset_index(drop=True)
    print("\n--- MIQP positional summary (10 runs each) ---")
    print(summary_df)
    summary_df.to_csv("tsp_miqp_positional_summary_N1to10_10runs.csv", index=False)

    # (선택) 마지막 N의 per-run 기록도 저장
    df_runs.to_csv(f"tsp_miqp_positional_runs_N{N}_{runs}runs.csv", index=False)


[Single run] N=7, n_point=8


MIQP runs:   0%|                                                                                | 0/10 [00:00<?, ?it/s]

ok: True distance: 265.2444588464126 time_ms: 121.48
tour: [0, 5, 6, 2, 3, 4, 1, 7]



N loop (summaries): 100%|██████████████████████████████████████████████████████████████| 10/10 [00:22<00:00,  2.24s/it]


--- MIQP positional summary (10 runs each) ---
    N  n_point  distance_mean  distance_std  time_mean_ms  time_std_ms  \
0   1        2     163.682620  2.995911e-14     15.260625     2.523784   
1   2        3     206.911077  0.000000e+00     12.037873     3.087217   
2   3        4     238.059641  2.995911e-14     32.951808     2.902608   
3   4        5     249.433830  2.320623e-14     47.221780     5.237386   
4   5        6     271.749465  5.991822e-14    121.402955    47.350985   
5   6        7     327.266489  5.991822e-14    175.054622    79.830392   
6   7        8     265.244459  5.991822e-14    111.866498    10.826404   
7   8        9     305.843631  5.684342e-14    196.088028    27.663299   
8   9       10     342.852479  5.359249e-14    544.942522   105.846803   
9  10       11     297.391534  5.991822e-14    657.125497    79.946817   

   feasible_rate  
0            1.0  
1            1.0  
2            1.0  
3            1.0  
4            1.0  
5            1.0  
6   